# Function

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
def Black_Schole(spot_price, strike_price, dividend_yield, volatility ,risk_free_rate,time_to_maturity):
  d1 = (np.log(spot_price / strike_price) + (dividend_yield + (volatility ** 2) / 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
  d2 = d1 - volatility * np.sqrt(time_to_maturity)
  N_d1 = norm.cdf(d1)
  N_d2 = norm.cdf(d2)
  call_option_price = spot_price * np.exp((dividend_yield - risk_free_rate) * time_to_maturity) * norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(d2)
  call_option_delta = np.exp((dividend_yield - risk_free_rate) * time_to_maturity) * norm.cdf(d1)
  put_option_price = spot_price * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(-d2) - strike_price*np.exp((dividend_yield-risk_free_rate)*time_to_maturity) * norm.cdf(-d1)
  put_option_delta = -np.exp((dividend_yield-risk_free_rate)*time_to_maturity) * norm.cdf(-d1)
  return d1, d2, N_d1, N_d2, call_option_price, call_option_delta, put_option_price, put_option_delta


In [3]:
import numpy as np

def generate_asset_price(spot_price, d, u, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if i == 0 and j == 0:
                x[i, j] = spot_price
            elif i == j:
                x[i, j] = x[i-1, j-1] * d
            elif i < j:
                x[i, j] = x[i, j-1] * u
            else:
                x[i, j] = 0

    return x

In [4]:
import numpy as np

def generate_european_call(spot_price, strike_price, d, u, q, r, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))
    y = generate_asset_price(spot_price, d, u, num_steps)

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if j == num_steps:
                x[i, j] = max(0, y[i, j] - strike_price)

    n = 1
    for k in range(num_steps + 1 - n):
        for i in range(num_steps -n +1):
            for j in range(num_steps):
                if j <= num_steps - 1 and i <= num_steps - 1and i <= j:
                    x[i, j] = (q * x[i, j+1] + (1 - q) * x[i+1, j+1]) / r
        n += 1
    return x

In [5]:
import numpy as np

def generate_american_call(spot_price, strike_price, d, u, q, r, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))
    y = generate_asset_price(spot_price, d, u, num_steps)

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if j == num_steps:
                x[i, j] = max(0, y[i, j] - strike_price)

    n = 1
    for k in range(num_steps + 1 - n):
        for i in range(num_steps -n + 1):
            for j in range(num_steps):
                if j <= num_steps - 1 and i <= num_steps - 1 and i <= j:
                    x[i, j] = np.maximum(y[i,j] -  strike_price,(q * x[i, j+1] + (1 - q) * x[i+1, j+1]) / r)

        n += 1
    return x

In [6]:
import numpy as np

def generate_european_put(spot_price, strike_price, d, u, q, r, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))
    y = generate_asset_price(spot_price, d, u, num_steps)

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if j == num_steps:
                x[i, j] = max(0, strike_price - y[i, j])

    n = 1
    for k in range(num_steps + 1 - n):
        for i in range(num_steps -n +1):
            for j in range(num_steps):
                if j <= num_steps - 1 and i <= num_steps - 1and i <= j:
                    x[i, j] = (q * x[i, j+1] + (1 - q) * x[i+1, j+1]) / r
        n += 1
    return x

In [7]:
import numpy as np


def generate_american_put(spot_price, strike_price, d, u, q, r, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))
    y = generate_asset_price(spot_price, d, u, num_steps)

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if j == num_steps:
                x[i, j] = max(0, strike_price - y[i, j])

    n = 1
    for k in range(num_steps + 1 - n):
        for i in range(num_steps -n + 1):
            for j in range(num_steps):
                if j <= num_steps - 1 and i <= num_steps - 1 and i <= j:
                    x[i, j] = np.maximum(strike_price - y[i,j],(q * x[i, j+1] + (1 - q) * x[i+1, j+1]) / r)

        n += 1
    return x

In [8]:
import numpy as np
from scipy.stats import norm

def monte_carlo_sim_option(spot_price, risk_free_rate, dividend_yield, volatility, time_to_maturity, num_paths):
  simulation = np.zeros((num_paths + 1, 1))

  for i in range(1, num_paths + 1):
    drift = (risk_free_rate - dividend_yield - 0.5 * volatility**2) * time_to_maturity
    vol_term = volatility * np.sqrt(time_to_maturity)
    random_number = np.random.rand()
    sim_price = spot_price * np.exp(drift + vol_term * norm.ppf(random_number))

    simulation[i-1, 0] = sim_price

  return simulation

# Run

In [20]:
import numpy as np
import pandas as pd
from scipy.stats import norm
# Example usage:
spot_price = 3500   # Current stock price
strike_price = 3500   # Option strike price
time_to_maturity = 3  # Time to option expiration in years
volatility = 0.24   # Volatility of the underlying stock
risk_free_rate = 0.045 # Risk-free interest rate
dividend_yield =  risk_free_rate - 0


In [21]:
d1, d2, N_d1, N_d2, call_option_price, call_option_delta, put_option_price, put_option_delta = Black_Schole(spot_price, strike_price, dividend_yield, risk_free_rate, volatility, time_to_maturity)

In [22]:
call_option_price = 400
Black_Schole(spot_price, strike_price, dividend_yield, risk_free_rate, volatility, time_to_maturity)

(1.7710219507391771,
 1.6930796643985777,
 0.9617214742156264,
 0.9547798477539339,
 248.63799056888865,
 0.535780670716171,
 2.4003700861808,
 -0.021325191096003008)

In [23]:
print(f"d1 is {d1:.3f}\nd2 is {d2:.3f}")
print(f"N(d1) is {N_d1:.3f}\nN(d2) is {N_d2:.3f}")
print(f"Call Value is {call_option_price:.3f}\nCall Delta is {call_option_delta:.3f}")
print(f"Put Value is {put_option_price:.3f}\nPut Delta is {put_option_delta:.3f}")

d1 is 1.771
d2 is 1.693
N(d1) is 0.962
N(d2) is 0.955
Call Value is 400.000
Call Delta is 0.536
Put Value is 2.400
Put Delta is -0.021


In [24]:
d1, d2, N_d1, N_d2, call_option_price, call_option_delta, put_option_price, put_option_delta = Black_Schole(spot_price, strike_price, dividend_yield, risk_free_rate, volatility, time_to_maturity)

In [25]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve

def implied_parameter_calculator(parameter_value, variable, target_option_price, *args):
    # Unpack args correctly
    spot_price, strike_price, dividend_yield, time_to_maturity, risk_free_rate, volatility, option_type = args

    # Set the parameter value based on the variable being solved for
    if variable == 'spot_price':
        spot_price = parameter_value
    elif variable == 'strike_price':
        strike_price = parameter_value
    elif variable == 'dividend_yield':
        dividend_yield = parameter_value
    elif variable == 'time_to_maturity':
        time_to_maturity = parameter_value
    elif variable == 'risk_free_rate':
        risk_free_rate = parameter_value
    else:
        raise ValueError(f"Invalid variable: {variable}")

    # Black-Scholes calculations
    d1 = (np.log(spot_price / strike_price) + (dividend_yield + (volatility ** 2) / 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
    d2 = d1 - volatility * np.sqrt(time_to_maturity)

    if option_type == 'call':
        option_price = spot_price * np.exp((dividend_yield - risk_free_rate) * time_to_maturity) * norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = strike_price * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(-d2) - spot_price * np.exp((dividend_yield - risk_free_rate) * time_to_maturity) * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option_type. Use 'call' or 'put'.")

    return option_price - target_option_price



# Target option price
target_option_price = 400

# Choose variable to solve for
variable_to_solve = 'risk_free_rate'

# Initial guess for the parameter value
initial_guess = risk_free_rate  # or any other appropriate initial guess

# Using fsolve to find the implied parameter value
implied_parameter_result = fsolve(implied_parameter_calculator, initial_guess, args=(variable_to_solve, target_option_price, spot_price, strike_price, dividend_yield, time_to_maturity, risk_free_rate, volatility, 'call'))

print(f"Implied {variable_to_solve}:", implied_parameter_result[0])


Implied risk_free_rate: 0.2712878171299127


In [30]:
import numpy as np
"""
spot_price = 274      # Current stock price
strike_price = 225   # Option strike price
time_to_maturity = 32/252  # Time to option expiration in years
volatility = 0.52    # Volatility of the underlying stock
risk_free_rate = 0.01 # Risk-free interest rate
dividend_yield = 0.00
"""
num_steps = 36


delta_t = time_to_maturity / num_steps
u = np.exp(volatility * np.sqrt(delta_t))
d = 1 / u
r = np.exp(risk_free_rate*delta_t)
b = np.exp(dividend_yield*delta_t)
q = (b-d)/(u-d)

In [31]:
Asset_Price = generate_asset_price(spot_price, d, u, num_steps)
Asset_Price_df = pd.DataFrame(Asset_Price)
Asset_Price_df = Asset_Price_df.round(2)
Asset_Price_df.replace(0, '', inplace=True)
Asset_Price_df.head(num_steps+1)
Asset_Price_df

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,3500.0,3751.08,4020.18,4308.58,4617.67,4948.94,5303.97,5684.47,6092.26,6529.31,...,22723.0,24353.12,26100.17,27972.56,29979.26,32129.93,34434.88,36905.19,39552.71,42390.15
1,,3265.72,3500.0,3751.08,4020.18,4308.58,4617.67,4948.94,5303.97,5684.47,...,19782.82,21202.01,22723.0,24353.12,26100.17,27972.56,29979.26,32129.93,34434.88,36905.19
2,,,3047.13,3265.72,3500.0,3751.08,4020.18,4308.58,4617.67,4948.94,...,17223.07,18458.63,19782.82,21202.01,22723.0,24353.12,26100.17,27972.56,29979.26,32129.93
3,,,,2843.16,3047.13,3265.72,3500.0,3751.08,4020.18,4308.58,...,14994.53,16070.22,17223.07,18458.63,19782.82,21202.01,22723.0,24353.12,26100.17,27972.56
4,,,,,2652.85,2843.16,3047.13,3265.72,3500.0,3751.08,...,13054.35,13990.85,14994.53,16070.22,17223.07,18458.63,19782.82,21202.01,22723.0,24353.12
5,,,,,,2475.28,2652.85,2843.16,3047.13,3265.72,...,11365.22,12180.54,13054.35,13990.85,14994.53,16070.22,17223.07,18458.63,19782.82,21202.01
6,,,,,,,2309.59,2475.28,2652.85,2843.16,...,9894.64,10604.47,11365.22,12180.54,13054.35,13990.85,14994.53,16070.22,17223.07,18458.63
7,,,,,,,,2155.0,2309.59,2475.28,...,8614.35,9232.33,9894.64,10604.47,11365.22,12180.54,13054.35,13990.85,14994.53,16070.22
8,,,,,,,,,2010.75,2155.0,...,7499.72,8037.74,8614.35,9232.33,9894.64,10604.47,11365.22,12180.54,13054.35,13990.85
9,,,,,,,,,,1876.16,...,6529.31,6997.71,7499.72,8037.74,8614.35,9232.33,9894.64,10604.47,11365.22,12180.54


In [32]:
euro_call = generate_european_call(spot_price, strike_price ,d, u, q, r ,num_steps)
euro_call_df = pd.DataFrame(euro_call)
euro_call_df = euro_call_df.round(2)
euro_call_df.replace(0, '', inplace=True)
euro_call_df.head(num_steps+1)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,784.75,954.64,1150.94,1375.4,1629.46,1914.32,2230.88,2579.92,2962.11,3378.18,...,19339.16,20956.56,22690.85,24550.43,26544.28,28682.04,30974.04,33431.34,36065.81,38890.15
1,,614.1,757.82,926.35,1121.73,1345.73,1599.79,1885.02,2202.26,2552.14,...,16398.97,17805.45,19313.68,20930.99,22665.19,24524.67,26518.42,28656.08,30947.98,33405.19
2,,,469.36,588.37,730.27,897.39,1091.86,1315.44,1569.57,1855.29,...,13839.22,15062.07,16373.5,17779.88,19288.02,20905.23,22639.33,24498.71,26492.37,28629.93
3,,,,349.2,445.32,562.07,702.05,867.72,1061.26,1284.48,...,11610.69,12673.66,13813.75,15036.5,16347.83,17754.11,19262.16,20879.27,22613.27,24472.56
4,,,,,251.93,327.33,420.81,535.15,673.11,837.26,...,9670.51,10594.29,11585.21,12648.09,13788.08,15010.73,16321.97,17728.16,19236.11,20853.12
5,,,,,,175.45,232.63,305.13,395.8,507.57,...,7981.37,8783.98,9645.03,10568.72,11559.55,12622.33,13762.22,14984.78,16295.92,17702.01
6,,,,,,,117.33,159.02,213.19,282.59,...,6510.8,7207.91,7955.9,8758.41,9619.37,10542.96,11533.69,12596.37,13736.17,14958.63
7,,,,,,,,74.88,103.91,142.65,...,5230.5,5835.77,6485.32,7182.34,7930.23,8732.65,9593.51,10517.0,11507.63,12570.22
8,,,,,,,,,45.26,64.43,...,4115.87,4641.18,5205.03,5810.2,6459.66,7156.58,7904.37,8706.69,9567.45,10490.85
9,,,,,,,,,,25.67,...,3145.46,3601.15,4090.4,4615.61,5179.36,5784.44,6433.8,7130.62,7878.32,8680.54


In [33]:
am_call = generate_american_call(spot_price, strike_price ,d, u, q, r ,num_steps)
am_call_df = pd.DataFrame(am_call)
am_call_df = am_call_df.round(2)
am_call_df.replace(0, '', inplace=True)
am_call_df.head(num_steps+1)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,784.75,954.64,1150.94,1375.4,1629.46,1914.32,2230.88,2579.92,2962.11,3378.18,...,19339.16,20956.56,22690.85,24550.43,26544.28,28682.04,30974.04,33431.34,36065.81,38890.15
1,,614.1,757.82,926.35,1121.73,1345.73,1599.79,1885.02,2202.26,2552.14,...,16398.97,17805.45,19313.68,20930.99,22665.19,24524.67,26518.42,28656.08,30947.98,33405.19
2,,,469.36,588.37,730.27,897.39,1091.86,1315.44,1569.57,1855.29,...,13839.22,15062.07,16373.5,17779.88,19288.02,20905.23,22639.33,24498.71,26492.37,28629.93
3,,,,349.2,445.32,562.07,702.05,867.72,1061.26,1284.48,...,11610.69,12673.66,13813.75,15036.5,16347.83,17754.11,19262.16,20879.27,22613.27,24472.56
4,,,,,251.93,327.33,420.81,535.15,673.11,837.26,...,9670.51,10594.29,11585.21,12648.09,13788.08,15010.73,16321.97,17728.16,19236.11,20853.12
5,,,,,,175.45,232.63,305.13,395.8,507.57,...,7981.37,8783.98,9645.03,10568.72,11559.55,12622.33,13762.22,14984.78,16295.92,17702.01
6,,,,,,,117.33,159.02,213.19,282.59,...,6510.8,7207.91,7955.9,8758.41,9619.37,10542.96,11533.69,12596.37,13736.17,14958.63
7,,,,,,,,74.88,103.91,142.65,...,5230.5,5835.77,6485.32,7182.34,7930.23,8732.65,9593.51,10517.0,11507.63,12570.22
8,,,,,,,,,45.26,64.43,...,4115.87,4641.18,5205.03,5810.2,6459.66,7156.58,7904.37,8706.69,9567.45,10490.85
9,,,,,,,,,,25.67,...,3145.46,3601.15,4090.4,4615.61,5179.36,5784.44,6433.8,7130.62,7878.32,8680.54


In [34]:
euro_put = generate_european_put(spot_price, strike_price ,d, u, q, r ,num_steps)
euro_put_df = pd.DataFrame(euro_put)
euro_put_df = euro_put_df.round(2)
euro_put_df.replace(0, '', inplace=True)
euro_put_df.head(num_steps+1)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,342.76,273.05,211.78,159.42,116.01,81.26,54.51,34.79,20.98,11.85,...,,,,,,,,,,
1,,417.88,338.84,267.87,205.77,153.04,109.71,75.42,49.42,30.65,...,,,,,,,,,,
2,,,503.26,415.25,334.49,262.19,199.27,146.2,103.03,69.33,...,,,,,,,,,,
3,,,,598.64,502.42,412.23,329.64,255.97,192.21,138.87,...,,,,,,,,,,
4,,,,,703.3,600.05,501.28,408.77,324.25,249.15,...,,,,,,,,,,
5,,,,,,816.05,707.37,601.31,499.8,404.82,...,,,,,,,,,,
6,,,,,,,935.33,823.08,711.47,602.41,...,,,,,,,,,,
7,,,,,,,,1059.22,945.46,830.34,...,,,,,,,,,,
8,,,,,,,,,1185.65,1072.4,...,,,,,,,,,,
9,,,,,,,,,,1312.49,...,,,,,,,,,,


In [35]:
am_put = generate_american_put(spot_price, strike_price ,d, u, q, r ,num_steps)
am_put_df = pd.DataFrame(am_put)
am_put_df = am_put_df.round(2)
am_put_df.replace(0, '', inplace=True)
am_put_df.head(num_steps+1)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,401.1,314.43,240.46,178.78,128.72,89.34,59.46,37.71,22.61,12.71,...,,,,,,,,,,
1,,494.29,393.76,306.44,232.2,170.66,121.1,82.54,53.69,33.09,...,,,,,,,,,,
2,,,602.62,487.59,385.99,297.99,223.5,162.13,113.17,75.53,...,,,,,,,,,,
3,,,,726.87,596.98,480.46,377.73,289.03,214.3,153.17,...,,,,,,,,,,
4,,,,,867.51,722.72,590.97,472.87,368.95,279.5,...,,,,,,,,,,
5,,,,,,1024.72,865.26,718.3,584.57,464.79,...,,,,,,,,,,
6,,,,,,,1190.41,1024.72,862.88,713.6,...,,,,,,,,,,
7,,,,,,,,1345.0,1190.41,1024.72,...,,,,,,,,,,
8,,,,,,,,,1489.25,1345.0,...,,,,,,,,,,
9,,,,,,,,,,1623.84,...,,,,,,,,,,


In [36]:
euro_call_binomial = euro_call_df[0][0]
am_call_binomial = am_call_df[0][0]
euro_put_binomial = euro_put_df[0][0]
am_put_binomial = am_put_df[0][0]
print(f"Binomial European Call Value: {euro_call_binomial: .3f}\nBinomial American Call Value: {am_call_binomial: .3f}")
print(f"Binomial European Put Value: {euro_put_binomial: .3f}\nBinomial American Put Value: {am_put_binomial: .3f}")

Binomial European Call Value:  784.750
Binomial American Call Value:  784.750
Binomial European Put Value:  342.760
Binomial American Put Value:  401.100


In [37]:
"""
spot_price = 274      # Current stock price
strike_price = 225   # Option strike price
time_to_maturity = 32/252  # Time to option expiration in years
volatility = 0.52    # Volatility of the underlying stock
risk_free_rate = 0.01 # Risk-free interest rate
dividend_yield = 0.00
num_steps = 10
"""

'\nspot_price = 274      # Current stock price\nstrike_price = 225   # Option strike price\ntime_to_maturity = 32/252  # Time to option expiration in years\nvolatility = 0.52    # Volatility of the underlying stock\nrisk_free_rate = 0.01 # Risk-free interest rate\ndividend_yield = 0.00\nnum_steps = 10\n'

In [38]:
num_paths = 10000
simulation_price = monte_carlo_sim_option(spot_price, risk_free_rate, dividend_yield, volatility, time_to_maturity, num_paths)
Call_Expired_Value = np.maximum(0, simulation_price - strike_price)
Call_Option =  np.exp((dividend_yield -risk_free_rate)*time_to_maturity) * Call_Expired_Value.mean()
Put_Expired_Value = np.maximum(0, strike_price - simulation_price)
Put_Option =  np.exp((dividend_yield -risk_free_rate)*time_to_maturity) * Put_Expired_Value.mean()
print(f"Monte Carlo call value is {Call_Option:.2f}\nMonte Carlo put value is {Put_Option:.2f}")

Monte Carlo call value is 583.41
Monte Carlo put value is 570.01


In [39]:
import numpy as np

# Monte Carlo simulation
def monte_carlo(spot_price, strike_price, dividend_yield ,risk_free_rate, volatility, time_to_maturity, num_steps):
    dt = time_to_maturity/ num_steps # Time step
    S = np.empty(num_steps+1) # Stock price
    S[0] = spot_price
    for i in range(num_steps):
        S[i+1] = S[i] * np.exp((r - dividend_yield - volatility**2 / 2) * dt + volatility * np.sqrt(dt) * np.random.normal())
    return np.maximum(S[-1] - strike_price, 0) # Payoff of call option


# Run Monte Carlo simulation
M = 1000 # Number of simulations
payoffs = np.empty(M)
for i in range(M):
    payoffs[i] = monte_carlo(spot_price, strike_price, dividend_yield ,risk_free_rate, volatility, time_to_maturity, num_steps)

# Estimate option price
C = np.exp(-r * time_to_maturity) * payoffs.mean()

print(C)

2882.732700100306
